In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [2]:
## Extracting features of 2023 movies from Wikipedia

In [3]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2023"

In [4]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [5]:
tables = soup.find_all('table',class_='wikitable sortable')

In [6]:
len(tables)

4

In [7]:
type(tables[0])

bs4.element.Tag

In [8]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [9]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14200\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [10]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,6,M3GAN,Universal Pictures / Blumhouse Productions / A...,Gerard Johnstone (director); Akela Cooper (scr...,[3]
1,J A N U A R Y,6,The Old Way,Saban Films / Saturn Films,Brett Donowho (director); Carl W. Lucas (scree...,[4]
2,J A N U A R Y,11,The Devil Conspiracy,Samuel Goldwyn Films,Nathan Frankowski (director); Ed Alan (screenp...,[5]
3,J A N U A R Y,13,Plane,Lionsgate / MadRiver Pictures / Di Bonaventura...,Jean-François Richet (director); Charles Cummi...,[6]
4,J A N U A R Y,13,House Party,Warner Bros. Pictures / New Line Cinema,"Calmatic (director); Jamal Olori, Stephen Glov...",[7]
...,...,...,...,...,...,...
201,D E C E M B E R,20,Aquaman and the Lost Kingdom,Warner Bros. Pictures / DC Studios / Atomic Mo...,James Wan (director); David Leslie Johnson-McG...,[179]
202,D E C E M B E R,20,Untitled Ghostbusters: Afterlife sequel,Columbia Pictures / Bron Creative / Ghost Corps,Gil Kenan (director/screenplay); Jason Reitman...,[180]
203,D E C E M B E R,22,Rebel Moon,Netflix / The Stone Quarry,Zack Snyder (director); Shay Hatten (screenpla...,[96]
204,D E C E M B E R,22,Migration,Universal Pictures / Illumination,Benjamin Renner (director); Mike White (screen...,[181]


In [11]:
df_2023 = df[['Title','Cast and crew']]

In [12]:
df_2023

,Title,Cast and crew
0,M3GAN,Gerard Johnstone (director); Akela Cooper (scr...
1,The Old Way,Brett Donowho (director); Carl W. Lucas (scree...
2,The Devil Conspiracy,Nathan Frankowski (director); Ed Alan (screenp...
3,Plane,Jean-François Richet (director); Charles Cummi...
4,House Party,"Calmatic (director); Jamal Olori, Stephen Glov..."
...,...,...
201,Aquaman and the Lost Kingdom,James Wan (director); David Leslie Johnson-McG...
202,Untitled Ghostbusters: Afterlife sequel,Gil Kenan (director/screenplay); Jason Reitman...
203,Rebel Moon,Zack Snyder (director); Shay Hatten (screenpla...
204,Migration,Benjamin Renner (director); Mike White (screen...


In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'ff3132c29c92e21e14539982f56e664a'

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [15]:
df_2023['genres'] = df_2023['Title'].map(lambda x: get_genre(str(x)))

C:\Users\Hp\AppData\Local\Temp\ipykernel_14200\2917389152.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['genres'] = df_2023['Title'].map(lambda x: get_genre(str(x)))


In [17]:
df_2023

,Title,Cast and crew,genres
0,M3GAN,Gerard Johnstone (director); Akela Cooper (scr...,Science Fiction Horror Comedy
1,The Old Way,Brett Donowho (director); Carl W. Lucas (scree...,Western
2,The Devil Conspiracy,Nathan Frankowski (director); Ed Alan (screenp...,Horror Fantasy Science Fiction Thriller
3,Plane,Jean-François Richet (director); Charles Cummi...,Action Adventure Thriller
4,House Party,"Calmatic (director); Jamal Olori, Stephen Glov...",Comedy
...,...,...,...
201,Aquaman and the Lost Kingdom,James Wan (director); David Leslie Johnson-McG...,Action Adventure Fantasy
202,Untitled Ghostbusters: Afterlife sequel,Gil Kenan (director/screenplay); Jason Reitman...,Fantasy Adventure Comedy
203,Rebel Moon,Zack Snyder (director); Shay Hatten (screenpla...,Science Fiction Fantasy Action
204,Migration,Benjamin Renner (director); Mike White (screen...,Animation Comedy Adventure


In [18]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [19]:
df_2023['director_name'] = df_2023['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\Hp\AppData\Local\Temp\ipykernel_14200\1551818249.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['director_name'] = df_2023['Cast and crew'].map(lambda x: get_director(str(x)))


In [20]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [21]:
df_2023['actor_1_name'] = df_2023['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\Hp\AppData\Local\Temp\ipykernel_14200\4038699429.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['actor_1_name'] = df_2023['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [22]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [23]:
df_2023['actor_2_name'] = df_2023['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [24]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [25]:
df_2023['actor_3_name'] = df_2023['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [26]:
df_2023

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,M3GAN,Gerard Johnstone (director); Akela Cooper (scr...,Science Fiction Horror Comedy,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald
1,The Old Way,Brett Donowho (director); Carl W. Lucas (scree...,Western,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,NaN
2,The Devil Conspiracy,Nathan Frankowski (director); Ed Alan (screenp...,Horror Fantasy Science Fiction Thriller,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall
3,Plane,Jean-François Richet (director); Charles Cummi...,Action Adventure Thriller,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An
4,House Party,"Calmatic (director); Jamal Olori, Stephen Glov...",Comedy,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom
...,...,...,...,...,...,...,...
201,Aquaman and the Lost Kingdom,James Wan (director); David Leslie Johnson-McG...,Action Adventure Fantasy,James Wan,Jason Momoa,Amber Heard,Willem Dafoe
202,Untitled Ghostbusters: Afterlife sequel,Gil Kenan (director/screenplay); Jason Reitman...,Fantasy Adventure Comedy,Gil Kenan,Mckenna Grace,Carrie Coon,Finn Wolfhard
203,Rebel Moon,Zack Snyder (director); Shay Hatten (screenpla...,Science Fiction Fantasy Action,Zack Snyder,Sofia Boutella,Charlie Hunnam,Ray Fisher
204,Migration,Benjamin Renner (director); Mike White (screen...,Animation Comedy Adventure,Benjamin Renner,Kumail Nanjiani,Elizabeth Banks,Keegan-Michael Key


In [27]:
df_2023 = df_2023.rename(columns={'Title':'movie_title'})

In [28]:
new_df23 = df_2023.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [29]:
new_df23

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror Comedy,M3GAN
1,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,NaN,Western,The Old Way
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,The Devil Conspiracy
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action Adventure Thriller,Plane
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,House Party
...,...,...,...,...,...,...
201,James Wan,Jason Momoa,Amber Heard,Willem Dafoe,Action Adventure Fantasy,Aquaman and the Lost Kingdom
202,Gil Kenan,Mckenna Grace,Carrie Coon,Finn Wolfhard,Fantasy Adventure Comedy,Untitled Ghostbusters: Afterlife sequel
203,Zack Snyder,Sofia Boutella,Charlie Hunnam,Ray Fisher,Science Fiction Fantasy Action,Rebel Moon
204,Benjamin Renner,Kumail Nanjiani,Elizabeth Banks,Keegan-Michael Key,Animation Comedy Adventure,Migration


In [30]:
new_df23['comb'] = new_df23['actor_1_name'] + ' ' + new_df23['actor_2_name'] + ' '+ new_df23['actor_3_name'] + ' '+ new_df23['director_name'] +' ' + new_df23['genres']

In [31]:
new_df23.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      1
actor_3_name      9
genres            1
movie_title       0
comb             10
dtype: int64

In [32]:
new_df23 = new_df23.dropna(how='any')

In [33]:
new_df23.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [34]:
new_df23['movie_title'] = new_df23['movie_title'].str.lower()

C:\Users\Hp\AppData\Local\Temp\ipykernel_14200\3055323516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df23['movie_title'] = new_df23['movie_title'].str.lower()


In [35]:
new_df23

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror Comedy,m3gan,Allison Williams Violet McGraw Amie Donald Ger...
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,the devil conspiracy,Alice Orr-Ewing Joe Doyle Eveline Hall Nathan ...
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action Adventure Thriller,plane,Gerard Butler Mike Colter Yoson An Jean-Franço...
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,house party,Tosin Cole Jacob Latimore Karen Obilom Calmati...
5,John Hyams,Gideon Adlon,Bethlehem Million,Marc Menchaca,Horror Thriller,sick,Gideon Adlon Bethlehem Million Marc Menchaca J...
...,...,...,...,...,...,...,...
201,James Wan,Jason Momoa,Amber Heard,Willem Dafoe,Action Adventure Fantasy,aquaman and the lost kingdom,Jason Momoa Amber Heard Willem Dafoe James Wan...
202,Gil Kenan,Mckenna Grace,Carrie Coon,Finn Wolfhard,Fantasy Adventure Comedy,untitled ghostbusters: afterlife sequel,Mckenna Grace Carrie Coon Finn Wolfhard Gil Ke...
203,Zack Snyder,Sofia Boutella,Charlie Hunnam,Ray Fisher,Science Fiction Fantasy Action,rebel moon,Sofia Boutella Charlie Hunnam Ray Fisher Zack ...
204,Benjamin Renner,Kumail Nanjiani,Elizabeth Banks,Keegan-Michael Key,Animation Comedy Adventure,migration,Kumail Nanjiani Elizabeth Banks Keegan-Michael...


In [36]:
old_df = pd.read_csv('datasets/final_data.csv')

In [37]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5879,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Tom Holland Rashida Jones Nick Brun...
5880,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance History,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5881,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5882,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [38]:
final_df = old_df.append(new_df23,ignore_index=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14200\2740235243.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df23,ignore_index=True)


In [39]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6075,James Wan,Jason Momoa,Amber Heard,Willem Dafoe,Action Adventure Fantasy,aquaman and the lost kingdom,Jason Momoa Amber Heard Willem Dafoe James Wan...
6076,Gil Kenan,Mckenna Grace,Carrie Coon,Finn Wolfhard,Fantasy Adventure Comedy,untitled ghostbusters: afterlife sequel,Mckenna Grace Carrie Coon Finn Wolfhard Gil Ke...
6077,Zack Snyder,Sofia Boutella,Charlie Hunnam,Ray Fisher,Science Fiction Fantasy Action,rebel moon,Sofia Boutella Charlie Hunnam Ray Fisher Zack ...
6078,Benjamin Renner,Kumail Nanjiani,Elizabeth Banks,Keegan-Michael Key,Animation Comedy Adventure,migration,Kumail Nanjiani Elizabeth Banks Keegan-Michael...


In [40]:
final_df.to_csv('main_data.csv',index=False)